In [14]:
import os
import math
import json
import re
import random
from tqdm import tqdm
from items import Item
from sentence_transformers import SentenceTransformer
import pickle
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import load_dataset
import chromadb
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [15]:
load_dotenv()
hf_token = 'HF_TOKEN'

In [16]:
with open("train.pkl", "rb") as file:
    train = pickle.load(file)

print(train[0].prompt)

How much does this cost to the nearest dollar?

Delphi FG0166 Fuel Pump Module
Delphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7

Price is $227.00


In [17]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [5]:
vector_db = "products_vectorStore"
chroma_db = chromadb.PersistentClient(vector_db)

collection_name = "products"

In [6]:
collection_name_list = [collection.name for collection in chroma_db.list_collections()]
if collection_name in collection_name_list:
    chroma_db.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")
collection = chroma_db.create_collection(collection_name)

Deleted existing collection: products


In [18]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

vector1, vector2 = model.encode(['man', 'woman'])

# len(vector)
# len(vector[0])


In [19]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.2f}")

In [20]:
how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

Similarity between Java and C++ is 50.68
Similarity between Java and mug is 25.85
Similarity between Cup of Java and mug is 49.32


In [13]:
NUMBER_OF_PRODS = len(train)

for i in tqdm(range(0, NUMBER_OF_PRODS, 1000)):
    docs = [description(item) for item in train[i: i+1000]]
    doc_metadata = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    doc_ids = [f"doc_{j}" for j in range(i, i+len(docs))]
    vectors = model.encode(docs).astype(float).tolist()
    collection.add(
        embeddings=vectors,
        ids= doc_ids,
        documents= docs,
        metadatas= doc_metadata
    )

  0%|          | 0/400 [00:00<?, ?it/s]Insert of existing embedding ID: doc_0
Insert of existing embedding ID: doc_1
Insert of existing embedding ID: doc_2
Insert of existing embedding ID: doc_3
Insert of existing embedding ID: doc_4
Insert of existing embedding ID: doc_5
Insert of existing embedding ID: doc_6
Insert of existing embedding ID: doc_7
Insert of existing embedding ID: doc_8
Insert of existing embedding ID: doc_9
Insert of existing embedding ID: doc_10
Insert of existing embedding ID: doc_11
Insert of existing embedding ID: doc_12
Insert of existing embedding ID: doc_13
Insert of existing embedding ID: doc_14
Insert of existing embedding ID: doc_15
Insert of existing embedding ID: doc_16
Insert of existing embedding ID: doc_17
Insert of existing embedding ID: doc_18
Insert of existing embedding ID: doc_19
Insert of existing embedding ID: doc_20
Insert of existing embedding ID: doc_21
Insert of existing embedding ID: doc_22
Insert of existing embedding ID: doc_23
Insert of e